# 03 - Xarray y NetCDF

Abriremos el archivo sintético `data/wrfout_sample.nc` incluido en el repositorio.

In [ ]:
# Celda de instalación (Colab)
try:
    import google.colab
    !pip install -r https://raw.githubusercontent.com/jpantojac/wrf-course/main/requirements_colab.txt

except Exception:
    pass

In [ ]:
import xarray as xr
ds = xr.open_dataset('../data/wrfout_sample.nc')
print(ds)

## Selección y operaciones

In [ ]:
# Seleccionar T2 y calcular media espacial
T2 = ds['T2']
print('Dimensiones:', T2.dims)
print('Media global (K):', float(T2.mean()))
# Convertir a C
T2_c = T2 - 273.15
print('Media global (C):', float(T2_c.mean()))

In [ ]:
# Extraer una lon/lat específica (index-based)
val = T2.isel(time=0, lat=2, lon=2)
print('Valor en celda central (K):', float(val))